<a href="https://colab.research.google.com/github/shahriariit/Violence_Videos_Detection/blob/main/Violence_Detection_from_Movies_Images.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install opencv-python
!pip install tensorflow
!pip install moviepy

In [ ]:
from google.colab import drive
import cv2
import os
import math
import matplotlib.pyplot as plt
import numpy as np
import csv
from tensorflow.keras.preprocessing import image
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.preprocessing.image import img_to_array, load_img
from sklearn.utils import shuffle
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from sklearn.model_selection import train_test_split
from tensorflow.keras.layers import LSTM
from tensorflow.keras.applications import MobileNetV3Large
from tensorflow.keras.applications import VGG16
from tensorflow.keras.applications import VGG19
from tensorflow.keras.applications import ResNet50V2
from tensorflow.keras.applications import InceptionV3
from tensorflow.keras.applications import InceptionResNetV2
from tensorflow.keras.applications import DenseNet121
from tensorflow.keras.applications import DenseNet169
from tensorflow.keras.applications import DenseNet201
from tensorflow.keras.applications import Xception
from tensorflow.keras.models import Model
from tensorflow.keras.applications import Xception
from tensorflow.keras.applications import InceptionV3
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.layers import GlobalAveragePooling2D
from tensorflow.keras.optimizers import Adam
from sklearn.metrics import classification_report, accuracy_score, cohen_kappa_score, precision_score, recall_score, f1_score, roc_auc_score, matthews_corrcoef
from imblearn.metrics import geometric_mean_score
from imblearn.metrics import sensitivity_score
from imblearn.metrics import specificity_score

In [ ]:
drive.mount('/content/drive')
dataset_path = '/content/drive/MyDrive/MDataset'

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
def load_images_from_folder(folder, label):
    """Load images from a folder, preprocess, and assign labels."""
    images = []
    labels = []
    for filename in os.listdir(folder):
        img_path = os.path.join(folder, filename)
        # Load and preprocess image
        img = load_img(img_path, target_size=(224, 224))  # Resize image to 224x224
        img = img_to_array(img) / 255.0  # Normalize to [0, 1]
        images.append(img)
        labels.append(label)
    return np.array(images), np.array(labels)


def build_transfer_learning_model(base_model_fn, input_shape=(224, 224, 3), learning_rate=0.001):
    """
    Builds a binary classification model using a specified base model function.

    Parameters:
    - base_model_fn: a function from keras.applications (e.g., VGG16, ResNet50V2)
    - input_shape: input shape of the images
    - learning_rate: learning rate for the optimizer

    Returns:
    - Compiled Keras Model
    """
    # Load the base model without the top classification layer
    base_model = base_model_fn(weights='imagenet', include_top=False, input_shape=input_shape)

    # Freeze base model layers
    for layer in base_model.layers:
        layer.trainable = False

    # Add custom classification head
    x = base_model.output
    x = GlobalAveragePooling2D()(x)
    x = Dense(1024, activation='relu')(x)
    x = Dropout(0.5)(x)
    predictions = Dense(1, activation='sigmoid')(x)

    # Final model
    model = Model(inputs=base_model.input, outputs=predictions)

    # Compile model
    model.compile(optimizer=Adam(learning_rate=learning_rate),
                  loss='binary_crossentropy',
                  metrics=['accuracy'])

    return model

def evaluate(base_model, X_test, y_test, output_file="classification_metrics.csv"):
    # Predict probabilities

    with open(output_file, mode='w', newline='') as file:
        writer = csv.writer(file)

        # Write the header row
        writer.writerow(["Accuracy", "Cohen-Kappa", "Precision", "Recall", "F1-Score", "AUROC", "MCC", "Specificity", "G-Mean", "Type I Error","Type II Error"])

        y_probs = base_model.predict(X_test)
        y_pred = (y_probs > 0.5).astype(int).flatten()

        # Classification Report (includes precision, recall, F1-score)
        print("Classification Report:\n", classification_report(y_test, y_pred))

        # Individual Metrics
        accuracy = accuracy_score(y_test, y_pred)
        cohen_kappa = cohen_kappa_score(y_test, y_pred)
        precision = precision_score(y_test, y_pred, average='weighted')
        recall = recall_score(y_test, y_pred, average='weighted')
        f1 = f1_score(y_test, y_pred, average='weighted')
        auroc = roc_auc_score(y_test, y_probs, average='weighted')
        mcc = matthews_corrcoef(y_test, y_pred)
        specificity = specificity_score(y_test, y_pred, average='weighted')
        g_mean = geometric_mean_score(y_test, y_pred, average='weighted')
        type_i_error = 1 - g_mean
        type_ii_error = 1 - specificity


        writer.writerow([accuracy, cohen_kappa, precision, recall, f1, auroc, mcc, specificity, g_mean, type_i_error, type_ii_error])

        # Display results
        print("Accuracy:", accuracy)
        print("Cohen's Kappa:", cohen_kappa)
        print("Precision:", precision)
        print("Recall:", recall)
        print("F1-Score:", f1)
        print("AUROC:", auroc)
        print("Matthews Correlation Coefficient (MCC):", mcc)
        print("Specificity",specificity)
        print("Geometric Mean",g_mean)
        print("Type I Error",type_i_error)
        print("Type II Error",type_ii_error)

In [ ]:
# Paths to the frames
nonviolence_folder = '/content/drive/MyDrive/MDataset/noFights'
violence_folder = '/content/drive/MyDrive/MDataset/fights'

In [ ]:
# Load images and labels for each class
violence_images, violence_labels = load_images_from_folder(violence_folder, 1)  # Label 1 for Violence
nonviolence_images, nonviolence_labels = load_images_from_folder(nonviolence_folder, 0)  # Label 0 for Non-Violence

# Combine the datasets
X = np.concatenate((violence_images, nonviolence_images), axis=0)
y = np.concatenate((violence_labels, nonviolence_labels), axis=0)

# Shuffle the dataset
X, y = shuffle(X, y, random_state=42)

print(f"Dataset loaded and shuffled.")
print(f"Total samples: {len(y)}")
print(f"Shape of X: {X.shape}, Shape of y: {y.shape}")

Dataset loaded and shuffled.
Total samples: 428
Shape of X: (428, 224, 224, 3), Shape of y: (428,)


In [ ]:
# Split dataset into training and testin
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
model_vgg16 = build_transfer_learning_model(VGG16)
model_vgg16.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=6, batch_size=32)
accuracy_vgg16 = model_vgg16.evaluate(X_test, y_test, verbose=0)[1]
print(f"Accuracy: {accuracy_vgg16}")
evaluate(model_vgg16, X_test, y_test,"vgg16_classification.csv")

Epoch 1/6
11/11 ━━━━━━━━━━━━━━━━━━━━ 13s 672ms/step - accuracy: 0.6843 - loss: 0.5637 - val_accuracy: 0.9651 - val_loss: 0.2612
Epoch 2/6
11/11 ━━━━━━━━━━━━━━━━━━━━ 3s 211ms/step - accuracy: 0.9230 - loss: 0.2538 - val_accuracy: 0.9186 - val_loss: 0.1788
Epoch 3/6
11/11 ━━━━━━━━━━━━━━━━━━━━ 3s 213ms/step - accuracy: 0.9431 - loss: 0.1577 - val_accuracy: 0.9884 - val_loss: 0.0978
Epoch 4/6
11/11 ━━━━━━━━━━━━━━━━━━━━ 2s 211ms/step - accuracy: 0.9717 - loss: 0.1086 - val_accuracy: 0.9884 - val_loss: 0.0719
Epoch 5/6
11/11 ━━━━━━━━━━━━━━━━━━━━ 3s 213ms/step - accuracy: 0.9814 - loss: 0.0791 - val_accuracy: 0.9884 - val_loss: 0.0606
Epoch 6/6
11/11 ━━━━━━━━━━━━━━━━━━━━ 3s 218ms/step - accuracy: 0.9885 - loss: 0.0526 - val_accuracy: 0.9884 - val_loss: 0.0452
Accuracy: 0.9883720874786377
3/3 ━━━━━━━━━━━━━━━━━━━━ 1s 404ms/step
Classification Report:
               precision    recall  f1-score   support

           0       1.00      0.97      0.99        35
           1       0.98      1.00   

In [ ]:
model_vgg19 = build_transfer_learning_model(VGG19)
model_vgg19.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=6, batch_size=32)
accuracy_vgg19 = model_vgg19.evaluate(X_test, y_test, verbose=0)[1]
print(f"Accuracy: {accuracy_vgg19}")
evaluate(model_vgg19, X_test, y_test,"vgg19_classification.csv")

Epoch 1/6
11/11 ━━━━━━━━━━━━━━━━━━━━ 9s 625ms/step - accuracy: 0.6300 - loss: 0.6084 - val_accuracy: 0.8721 - val_loss: 0.3691
Epoch 2/6
11/11 ━━━━━━━━━━━━━━━━━━━━ 5s 258ms/step - accuracy: 0.9023 - loss: 0.3163 - val_accuracy: 0.9070 - val_loss: 0.2002
Epoch 3/6
11/11 ━━━━━━━━━━━━━━━━━━━━ 3s 264ms/step - accuracy: 0.9357 - loss: 0.2114 - val_accuracy: 0.9186 - val_loss: 0.1649
Epoch 4/6
11/11 ━━━━━━━━━━━━━━━━━━━━ 3s 262ms/step - accuracy: 0.9270 - loss: 0.1834 - val_accuracy: 0.9884 - val_loss: 0.1132
Epoch 5/6
11/11 ━━━━━━━━━━━━━━━━━━━━ 3s 261ms/step - accuracy: 0.9383 - loss: 0.1486 - val_accuracy: 1.0000 - val_loss: 0.0885
Epoch 6/6
11/11 ━━━━━━━━━━━━━━━━━━━━ 5s 256ms/step - accuracy: 0.9824 - loss: 0.1091 - val_accuracy: 1.0000 - val_loss: 0.0613
Accuracy: 1.0
2/3 ━━━━━━━━━━━━━━━━━━━━ 0s 190ms/step

3/3 ━━━━━━━━━━━━━━━━━━━━ 2s 485ms/step
Classification Report:
               precision    recall  f1-score   support

           0       1.00      1.00      1.00        35
           1       1.00      1.00      1.00        51

    accuracy                           1.00        86
   macro avg       1.00      1.00      1.00        86
weighted avg       1.00      1.00      1.00        86

Accuracy: 1.0
Cohen's Kappa: 1.0
Precision: 1.0
Recall: 1.0
F1-Score: 1.0
AUROC: 1.0
Matthews Correlation Coefficient (MCC): 1.0
Specificity 1.0
Geometric Mean 1.0
Type I Error 0.0
Type II Error 0.0


In [ ]:
model_mobilenetv3 = build_transfer_learning_model(MobileNetV3Large)
model_mobilenetv3.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=6, batch_size=32)
accuracy_mobilenetv3 = model_mobilenetv3.evaluate(X_test, y_test, verbose=0)[1]
print(f"Accuracy: {accuracy_mobilenetv3}")
evaluate(model_mobilenetv3, X_test, y_test,"mobilenetv3_classification.csv")

Epoch 1/6
11/11 ━━━━━━━━━━━━━━━━━━━━ 29s 2s/step - accuracy: 0.5278 - loss: 0.7011 - val_accuracy: 0.4070 - val_loss: 0.5934
Epoch 2/6
11/11 ━━━━━━━━━━━━━━━━━━━━ 20s 45ms/step - accuracy: 0.7335 - loss: 0.4801 - val_accuracy: 0.9884 - val_loss: 0.3092
Epoch 3/6
11/11 ━━━━━━━━━━━━━━━━━━━━ 0s 41ms/step - accuracy: 0.9063 - loss: 0.3418 - val_accuracy: 0.9419 - val_loss: 0.2143
Epoch 4/6
11/11 ━━━━━━━━━━━━━━━━━━━━ 1s 44ms/step - accuracy: 0.9615 - loss: 0.2422 - val_accuracy: 0.9884 - val_loss: 0.1645
Epoch 5/6
11/11 ━━━━━━━━━━━━━━━━━━━━ 0s 41ms/step - accuracy: 0.9617 - loss: 0.1956 - val_accuracy: 0.9884 - val_loss: 0.1328
Epoch 6/6
11/11 ━━━━━━━━━━━━━━━━━━━━ 1s 45ms/step - accuracy: 0.9590 - loss: 0.1694 - val_accuracy: 0.9884 - val_loss: 0.1151
Accuracy: 0.9883720874786377


3/3 ━━━━━━━━━━━━━━━━━━━━ 9s 2s/step
Classification Report:
               precision    recall  f1-score   support

           0       1.00      0.97      0.99        35
           1       0.98      1.00      0.99        51

    accuracy                           0.99        86
   macro avg       0.99      0.99      0.99        86
weighted avg       0.99      0.99      0.99        86

Accuracy: 0.9883720930232558
Cohen's Kappa: 0.975801913337085
Precision: 0.9885957066189625
Recall: 0.9883720930232558
F1-Score: 0.9883442789781446
AUROC: 0.9988795518207283
Matthews Correlation Coefficient (MCC): 0.9760877279974647
Specificity 0.9830564784053156
Geometric Mean 0.9857107025499585
Type I Error 0.014289297450041483
Type II Error 0.016943521594684374


In [ ]:
model_resnet50v2 = build_transfer_learning_model(ResNet50V2)
model_resnet50v2.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=6, batch_size=32)
accuracy_resnet50v2 = model_resnet50v2.evaluate(X_test, y_test, verbose=0)[1]
print(f"Accuracy: {accuracy_resnet50v2}")
evaluate(model_resnet50v2, X_test, y_test,"resnet50v2_classification.csv")

Epoch 1/6
11/11 ━━━━━━━━━━━━━━━━━━━━ 24s 1s/step - accuracy: 0.7591 - loss: 0.6497 - val_accuracy: 0.9535 - val_loss: 0.1742
Epoch 2/6
11/11 ━━━━━━━━━━━━━━━━━━━━ 7s 106ms/step - accuracy: 0.9627 - loss: 0.0819 - val_accuracy: 0.9767 - val_loss: 0.0297
Epoch 3/6
11/11 ━━━━━━━━━━━━━━━━━━━━ 1s 92ms/step - accuracy: 0.9919 - loss: 0.0160 - val_accuracy: 0.9884 - val_loss: 0.0340
Epoch 4/6
11/11 ━━━━━━━━━━━━━━━━━━━━ 1s 92ms/step - accuracy: 0.9945 - loss: 0.0077 - val_accuracy: 0.9884 - val_loss: 0.0249
Epoch 5/6
11/11 ━━━━━━━━━━━━━━━━━━━━ 1s 100ms/step - accuracy: 1.0000 - loss: 0.0016 - val_accuracy: 0.9884 - val_loss: 0.0244
Epoch 6/6
11/11 ━━━━━━━━━━━━━━━━━━━━ 1s 91ms/step - accuracy: 1.0000 - loss: 5.2552e-04 - val_accuracy: 0.9884 - val_loss: 0.0228
Accuracy: 0.9883720874786377
3/3 ━━━━━━━━━━━━━━━━━━━━ 8s 2s/step
Classification Report:
               precision    recall  f1-score   support

           0       0.97      1.00      0.99        35
           1       1.00      0.98      0.

In [ ]:
model_inceptionresnetv2 = build_transfer_learning_model(InceptionResNetV2)
model_inceptionresnetv2.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=6, batch_size=32)
accuracy_inceptionresnetv2 = model_inceptionresnetv2.evaluate(X_test, y_test, verbose=0)[1]
print(f"Accuracy: {accuracy_inceptionresnetv2}")
evaluate(model_inceptionresnetv2, X_test, y_test,"inceptionresnetv2_classification.csv")

Epoch 1/6
11/11 ━━━━━━━━━━━━━━━━━━━━ 68s 4s/step - accuracy: 0.7338 - loss: 0.7990 - val_accuracy: 0.8605 - val_loss: 0.2756
Epoch 2/6
11/11 ━━━━━━━━━━━━━━━━━━━━ 37s 191ms/step - accuracy: 0.8812 - loss: 0.2500 - val_accuracy: 0.9186 - val_loss: 0.1277
Epoch 3/6
11/11 ━━━━━━━━━━━━━━━━━━━━ 2s 200ms/step - accuracy: 0.9770 - loss: 0.0717 - val_accuracy: 0.9535 - val_loss: 0.0691
Epoch 4/6
11/11 ━━━━━━━━━━━━━━━━━━━━ 3s 219ms/step - accuracy: 0.9895 - loss: 0.0308 - val_accuracy: 0.9767 - val_loss: 0.0510
Epoch 5/6
11/11 ━━━━━━━━━━━━━━━━━━━━ 2s 193ms/step - accuracy: 0.9869 - loss: 0.0364 - val_accuracy: 0.9884 - val_loss: 0.0345
Epoch 6/6
11/11 ━━━━━━━━━━━━━━━━━━━━ 2s 193ms/step - accuracy: 0.9924 - loss: 0.0181 - val_accuracy: 0.9884 - val_loss: 0.0369
Accuracy: 0.9883720874786377
3/3 ━━━━━━━━━━━━━━━━━━━━ 26s 7s/step
Classification Report:
               precision    recall  f1-score   support

           0       0.97      1.00      0.99        35
           1       1.00      0.98      0

In [ ]:
model_densenet121 = build_transfer_learning_model(DenseNet121)
model_densenet121.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=6, batch_size=32)
accuracy_densenet121 = model_densenet121.evaluate(X_test, y_test, verbose=0)[1]
print(f"Accuracy: {accuracy_densenet121}")
evaluate(model_densenet121, X_test, y_test,"densenet121_classification.csv")

Epoch 1/6
11/11 ━━━━━━━━━━━━━━━━━━━━ 61s 3s/step - accuracy: 0.7887 - loss: 0.4573 - val_accuracy: 0.9767 - val_loss: 0.1077
Epoch 2/6
11/11 ━━━━━━━━━━━━━━━━━━━━ 40s 104ms/step - accuracy: 0.9280 - loss: 0.1437 - val_accuracy: 0.9884 - val_loss: 0.0509
Epoch 3/6
11/11 ━━━━━━━━━━━━━━━━━━━━ 1s 111ms/step - accuracy: 0.9900 - loss: 0.0490 - val_accuracy: 0.9651 - val_loss: 0.0663
Epoch 4/6
11/11 ━━━━━━━━━━━━━━━━━━━━ 1s 114ms/step - accuracy: 0.9808 - loss: 0.0489 - val_accuracy: 0.9884 - val_loss: 0.0560
Epoch 5/6
11/11 ━━━━━━━━━━━━━━━━━━━━ 2s 107ms/step - accuracy: 0.9918 - loss: 0.0270 - val_accuracy: 1.0000 - val_loss: 0.0249
Epoch 6/6
11/11 ━━━━━━━━━━━━━━━━━━━━ 1s 109ms/step - accuracy: 0.9917 - loss: 0.0160 - val_accuracy: 1.0000 - val_loss: 0.0169
Accuracy: 1.0
3/3 ━━━━━━━━━━━━━━━━━━━━ 24s 6s/step
Classification Report:
               precision    recall  f1-score   support

           0       1.00      1.00      1.00        35
           1       1.00      1.00      1.00        51



In [ ]:
model_densenet169 = build_transfer_learning_model(DenseNet169)
model_densenet169.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=6, batch_size=32)
accuracy_densenet169 = model_densenet169.evaluate(X_test, y_test, verbose=0)[1]
print(f"Accuracy: {accuracy_densenet169}")
evaluate(model_densenet169, X_test, y_test,"densenet169_classification.csv")

Epoch 1/6
11/11 ━━━━━━━━━━━━━━━━━━━━ 83s 5s/step - accuracy: 0.8170 - loss: 0.3522 - val_accuracy: 1.0000 - val_loss: 0.0184
Epoch 2/6
11/11 ━━━━━━━━━━━━━━━━━━━━ 21s 123ms/step - accuracy: 0.9905 - loss: 0.0238 - val_accuracy: 1.0000 - val_loss: 0.0144
Epoch 3/6
11/11 ━━━━━━━━━━━━━━━━━━━━ 54s 5s/step - accuracy: 1.0000 - loss: 0.0066 - val_accuracy: 0.9884 - val_loss: 0.0203
Epoch 4/6
11/11 ━━━━━━━━━━━━━━━━━━━━ 1s 119ms/step - accuracy: 1.0000 - loss: 0.0070 - val_accuracy: 1.0000 - val_loss: 0.0090
Epoch 5/6
11/11 ━━━━━━━━━━━━━━━━━━━━ 1s 127ms/step - accuracy: 1.0000 - loss: 0.0051 - val_accuracy: 1.0000 - val_loss: 0.0051
Epoch 6/6
11/11 ━━━━━━━━━━━━━━━━━━━━ 3s 134ms/step - accuracy: 1.0000 - loss: 0.0015 - val_accuracy: 1.0000 - val_loss: 0.0064
Accuracy: 1.0
3/3 ━━━━━━━━━━━━━━━━━━━━ 36s 9s/step
Classification Report:
               precision    recall  f1-score   support

           0       1.00      1.00      1.00        35
           1       1.00      1.00      1.00        51

  

In [ ]:
model_densenet201 = build_transfer_learning_model(DenseNet201)
model_densenet201.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=6, batch_size=32)
accuracy_densenet201 = model_densenet201.evaluate(X_test, y_test, verbose=0)[1]
print(f"Accuracy: {accuracy_densenet201}")
evaluate(model_densenet201, X_test, y_test,"densenet201_classification.csv")

Epoch 1/6


In [ ]:
model_inceptionv3 = build_transfer_learning_model(InceptionV3)
model_inceptionv3.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=6, batch_size=32)
accuracy_inceptionv3 = model_inceptionv3.evaluate(X_test, y_test, verbose=0)[1]
print(f"Accuracy: {accuracy_inceptionv3}")
evaluate(model_inceptionv3, X_test, y_test,"inceptionv3_classification.csv")